In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
path = "./../Dashboard"

# Generando data frame para los precios promedio

In [6]:
insumos = pd.read_excel(f"{path}/Libro1.xlsx", sheet_name="Insumos utilizados")
insumos.head()

,Historia clinica,Medicamento o Insumo,Precio,Cantidad
0,10001,ACEITE DE SILICONA 5000 POR 10ML SOLUCION OFTA...,235200,1
1,10002,ACEITE DE SILICONA 5000 POR 10ML SOLUCION OFTA...,235200,2
2,10003,ACEITE DE SILICONA 5000 POR 10ML SOLUCION OFTA...,275500,1
3,10004,ACEITE DE SILICONA PESADO POR 10ML OPIMED DE...,938588,1
4,10005,ACEITE MINERAL 450ML FRASCO DROFARMA,10557,1


In [27]:
insumos["precio_total"] = insumos["Precio"] * insumos["Cantidad"]

In [171]:
maxprecio_insumos = insumos.groupby("Medicamento o Insumo")["precio_total"].max().to_frame().reset_index().sort_values(by = "precio_total")
minprecio_insumos = insumos.groupby("Medicamento o Insumo")["precio_total"].min().to_frame().reset_index().sort_values(by = "precio_total")

In [52]:
maxprecio_insumos["Medicamento o Insumo"].sort_values().values == minprecio_insumos["Medicamento o Insumo"].sort_values().values

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True])

In [172]:
total_precios = maxprecio_insumos.merge(minprecio_insumos, on = "Medicamento o Insumo")
total_precios.rename(columns = {"precio_total_x": "preciomax", "precio_total_y": "preciomin"}, inplace = True)

In [173]:
total_precios

,Medicamento o Insumo,preciomax,preciomin
0,ACEITE MINERAL 500ML FRASCO LEON,10557,10557
1,ACEITE MINERAL 500ML FRASCO QUIMIBEN,10557,10557
2,ACEITE MINERAL 450ML FRASCO DROFARMA,21114,10557
3,ACEITE MINERAL 500ML FRASCO DISANFER,21114,10557
4,ACEITE MINERAL 500ML FRASCO QUIMICOS PROQUIMPI,21114,10557
5,ACEITE MINERAL 500ML FRASCO SAN JORGE,21114,10557
6,ACEITE MINERAL 500ML FRASCO OSA,31671,10557
7,ACEITE MINERAL 500ML FRASCO VERAXIPHARMA,31671,10557
8,ACEITE DE SILICONA 5000 POR 10ML SOLUCION OFTA...,235200,235200
9,ACEITE DE SILICONA 5000 POR 10ML SOLUCION OFTA...,275500,275500


In [174]:
total_precios.to_csv("ComparativaPrecios.csv", index=False)

# Generando data frame para costo por paciente discriminando por eps

In [66]:
costo_por_eps = pd.read_excel(f"{path}/Libro1.xlsx", sheet_name="Costos Promedio")
costo_por_eps.head()

,Diagnostico de Ingreso,Costo EPS 1,Costo EPS 2,Costo EPS 3,Costo EPS 4
0,AMPUTACION TRAUMATICA DE OTRO DEDO UNICO (COMP...,50500,55550.0,61105.0,85547.0
1,CONTUSION DE DEDO(S) DE LA MANO CON DAÑO DE LA...,28000,30800.0,33880.0,20328.0
2,CONTUSION DE LA CADERA,120000,126000.0,138600.0,194040.0
3,CONTUSION DE LA RODILLA,20000,22000.0,24200.0,33880.0
4,CONTUSION DE OTRAS PARTES Y DE LAS NO ESPECIFI...,30100,36120.0,39732.0,55624.8


In [69]:
costo_por_eps["Diagnostico de Ingreso"].nunique() == costo_por_eps.shape[0]

True

In [73]:
costo_melted = costo_por_eps.melt(id_vars= ["Diagnostico de Ingreso"]).sort_values(by = "Diagnostico de Ingreso")

In [74]:
costo_melted["variable"]

0     Costo EPS 1
34    Costo EPS 3
17    Costo EPS 2
51    Costo EPS 4
18    Costo EPS 2
         ...     
15    Costo EPS 1
33    Costo EPS 2
16    Costo EPS 1
50    Costo EPS 3
67    Costo EPS 4
Name: variable, Length: 68, dtype: object

In [76]:
costo_melted["variable"] = costo_melted["variable"].str.replace("Costo ", "")

In [78]:
costo_melted.rename(columns = {"variable": "EPS", "value": "Costo"}, inplace = True)

In [81]:
costo_melted

,Diagnostico de Ingreso,EPS,Costo
0,AMPUTACION TRAUMATICA DE OTRO DEDO UNICO (COMP...,EPS 1,50500.0
34,AMPUTACION TRAUMATICA DE OTRO DEDO UNICO (COMP...,EPS 3,61105.0
17,AMPUTACION TRAUMATICA DE OTRO DEDO UNICO (COMP...,EPS 2,55550.0
51,AMPUTACION TRAUMATICA DE OTRO DEDO UNICO (COMP...,EPS 4,85547.0
18,CONTUSION DE DEDO(S) DE LA MANO CON DAÑO DE LA...,EPS 2,30800.0
...,...,...,...
15,TRAUMATISMO DE LA CABEZA NO ESPECIFICADO,EPS 1,60000.0
33,TRAUMATISMOS MULTIPLES NO ESPECIFICADOS,EPS 2,53350.0
16,TRAUMATISMOS MULTIPLES NO ESPECIFICADOS,EPS 1,48500.0
50,TRAUMATISMOS MULTIPLES NO ESPECIFICADOS,EPS 3,58685.0


In [83]:
registros = pd.read_excel(f"{path}/Libro1.xlsx", sheet_name="Registro")
registros.head()

,Historia Clinica,sexo,diag_ingreso,Fecha Ingreso,Días internados
0,10001,F,HERIDA DE LA CABEZA PARTE NO ESPECIFICADA,2018-10-11,10
1,10002,M,HERIDA DE LA MUÑECA Y DE LA MANO PARTE NO ESPE...,2017-03-06,3
2,10003,M,HERIDA DE DEDO(S) DE LA MANO CON DAÑO DE LA(S)...,2018-10-12,10
3,10004,M,HERIDA DE DEDO(S) DE LA MANO CON DAÑO DE LA(S)...,2018-10-12,10
4,10005,M,AMPUTACION TRAUMATICA DE OTRO DEDO UNICO (COMP...,2017-03-06,3


In [85]:
diagnosticos_full = costo_melted.merge(registros, left_on= "Diagnostico de Ingreso", right_on= "diag_ingreso")

In [91]:
diagnosticos_full.isna().sum()

Diagnostico de Ingreso    0
EPS                       0
Costo                     0
Historia Clinica          0
sexo                      0
diag_ingreso              0
Fecha Ingreso             0
Días internados           0
dtype: int64

In [95]:
diagnosticos_full.sort_values(by = "Diagnostico de Ingreso,EPS".split(","), inplace=True)

In [97]:
diagnosticos_full.to_csv("TiempoPromedioDiagnosticos.csv", index=False)

In [107]:
diagnosticos_full["sexo"].unique().tolist()

['M', 'F']

In [183]:
diagnosticos_full.drop_duplicates(subset=["Diagnostico de Ingreso", "EPS", "sexo"]).to_csv("TiempoPromedioDiagnosticos.csv", index=False)

In [180]:
diagnosticos_full.shape

(104, 8)

In [108]:
import plotly.express as px

In [170]:
total_precios.sort_values(by = "Medicamento o Insumo")

,Medicamento o Insumo,preciomin,preciomax
8,ACEITE DE SILICONA 5000 POR 10ML SOLUCION OFTA...,235200,235200
10,ACEITE DE SILICONA 5000 POR 10ML SOLUCION OFTA...,470400,470400
9,ACEITE DE SILICONA 5000 POR 10ML SOLUCION OFTA...,275500,275500
11,ACEITE DE SILICONA PESADO POR 10ML OPIMED DE...,938588,938588
2,ACEITE MINERAL 450ML FRASCO DROFARMA,21114,10557
3,ACEITE MINERAL 500ML FRASCO DISANFER,21114,10557
0,ACEITE MINERAL 500ML FRASCO LEON,10557,10557
6,ACEITE MINERAL 500ML FRASCO OSA,31671,10557
1,ACEITE MINERAL 500ML FRASCO QUIMIBEN,10557,10557
4,ACEITE MINERAL 500ML FRASCO QUIMICOS PROQUIMPI,21114,10557


In [110]:
total_precios.columns

Index(['Medicamento o Insumo', 'preciomin', 'preciomax'], dtype='object')

In [168]:
ensayo

,Medicamento o Insumo,variable,value
0,ACEITE MINERAL 500ML FRASCO LEON,preciomin,10557
1,ACEITE MINERAL 500ML FRASCO QUIMIBEN,preciomin,10557
2,ACEITE MINERAL 450ML FRASCO DROFARMA,preciomin,21114
3,ACEITE MINERAL 500ML FRASCO DISANFER,preciomin,21114
4,ACEITE MINERAL 500ML FRASCO QUIMICOS PROQUIMPI,preciomin,21114
5,ACEITE MINERAL 500ML FRASCO SAN JORGE,preciomin,21114
6,ACEITE MINERAL 500ML FRASCO OSA,preciomin,31671
7,ACEITE MINERAL 500ML FRASCO VERAXIPHARMA,preciomin,31671
8,ACEITE DE SILICONA 5000 POR 10ML SOLUCION OFTA...,preciomin,235200
9,ACEITE DE SILICONA 5000 POR 10ML SOLUCION OFTA...,preciomin,275500


In [131]:
ensayo = total_precios.melt(id_vars = ["Medicamento o Insumo"])
fig = px.bar(ensayo, x = "Medicamento o Insumo", y = "value", color= "variable", barmode="group",
labels = {"value": "Costo", "variable": "Precio"}, title = "Comparativa de los precios de los medicamentos", )

In [139]:
fig.update_layout(xaxis = dict(showticklabels =  False))

# Sizas

In [146]:
diagnosticos_fullM = diagnosticos_full[(diagnosticos_full["sexo"] == "M") & (diagnosticos_full["EPS"] == "EPS 1")]

In [151]:
diagnosticos_fullM.columns

Index(['Diagnostico de Ingreso', 'EPS', 'Costo', 'Historia Clinica', 'sexo',
       'diag_ingreso', 'Fecha Ingreso', 'Días internados'],
      dtype='object')

In [152]:
diag_ensayo = diagnosticos_fullM[diagnosticos_fullM["Diagnostico de Ingreso"] == "AMPUTACION TRAUMATICA DE OTRO DEDO UNICO (COMPLETA) (PARCIAL"]

In [155]:
diag_ensayo

,Diagnostico de Ingreso,EPS,Costo,Historia Clinica,sexo,diag_ingreso,Fecha Ingreso,Días internados
0,AMPUTACION TRAUMATICA DE OTRO DEDO UNICO (COMP...,EPS 1,50500.0,10005,M,AMPUTACION TRAUMATICA DE OTRO DEDO UNICO (COMP...,2017-03-06,3


In [165]:
x = np.arange(1, diag_ensayo["Días internados"].values[0] + 1)
y = x.copy()*diag_ensayo["Costo"].values[0]
y = np.cumsum(y)

In [166]:
este_ultimo = pd.DataFrame(x, y)

In [177]:
sizas = px.line(este_ultimo, x = x, y = y)
sizas.add_bar(x = x, y = y, name = "Costo acumulado")
sizas.show()

In [175]:
px.bar(este_ultimo, x = x, y = y)